# Bowling Pins Problem
- Given a sequence of \(n\) pins, each with an integer value \(v_{i}\), a player can knock down pins in two ways:
    - Knock down a single pin \(i\), scoring \(v_{i}\) points.
    - Knock down two adjacent pins \(i\) and \(i+1\), scoring \(v_{i}\cdot v_{i+1}\) points.
- Each pin can only be knocked down once. 
- The objective is to find the maximum possible total score.

pin_values = [1, 2, -3, 4, 5]
Max value = 23
Explanation: Knock down 1: current total = 1, Knock down 2: current total = 3, skip -3: current total = 3, Knock down 4 and 5 together equals 4 * 5 = 20: current total = 23



## Solution
- Used dynamic programming to solve the problem

### 3 constraints or cases
- knock down 1 pin
- knock down none
- knock down 2 pins
- Get the max of each of those

### Recurrence relation
- the max of 3 constraints

### base case
- the max of 0 and the max of index 1 and 0

### topological ordering
- Just go through each index in the array

### memoization
- use a array to store the best selection at each indices






In [1]:
def max_bowl_pins(pins):
    if not pins:
        return 0

    # prepend a zero to pins array
    values = [0] + pins
    n = len(pins)

    memo = [0] * (n + 1)
    if n >= 1:
        memo[1] = max(values[1], 0)
    
    for i in range(2, n + 1):
        score1 = values[i] + memo[i - 1]
        score2 = (values[i -1] * values[i]) + memo[i - 2]
        score3 = memo[i - 1]

        memo[i] = max(score1, score2, score3)
    
    return memo[n]
    
    

In [2]:
pin_values = [1, 2, -3, 4, 5]
max_score = max_bowl_pins(pin_values)
print(f"The maximum score for the pin sequence {pin_values} is: {max_score}")

The maximum score for the pin sequence [1, 2, -3, 4, 5] is: 23


In [3]:
pin_values = [3, 4, -1, 6, -1, 6, 6, 3, -1, -1, 6, -2]
max_score = max_bowl_pins(pin_values)
print(f"The maximum score for the pin sequence {pin_values} is: {max_score}")

The maximum score for the pin sequence [3, 4, -1, 6, -1, 6, 6, 3, -1, -1, 6, -2] is: 64


## Now, turn the problem into a Direct Acyclic Graph (DAG)
- each pin is a node
- the constrains are the edges
- the goal is to use topological sort to find the max score to each node

In [4]:
def create_bowling_dag_adjacency(pins):
    """
    Creates an adjacency list representation of the bowling problem DAG.

    The graph nodes are 0, 1, ..., n. 
    Edges are (source, destination, weight).
    """
    n = len(pins)
    # Use 1-based indexing for pins internally
    v = [0] + pins
    
    # Adjacency list where keys are destination nodes, and values are 
    # a list of possible (source_node, score_gained) tuples.
    graph = {i: [] for i in range(n + 1)}
    
    # Node 0 is the start with 0 score.
    
    for i in range(1, n + 1):
        # Case 1: Edge from i-1 to i (Take pin i alone)
        graph[i].append((i - 1, v[i]))
        
        # Case 2: Edge from i-1 to i (Skip pin i) - weight 0
        graph[i].append((i - 1, 0))
        
        # Case 3: Edge from i-2 to i (Take pins i-1 and i as a pair)
        if i >= 2:
            graph[i].append((i - 2, v[i-1] * v[i]))
            
    return graph


In [5]:
pin_values = [1, 2, -3, 4, 5]
dag_graph = create_bowling_dag_adjacency(pin_values)

# Print the graph structure
for node, edges in dag_graph.items():
    print(f"Edges leading *to* Node {node}:")
    for source, weight in edges:
        print(f"  From Node {source} (Weight: {weight})")


Edges leading *to* Node 0:
Edges leading *to* Node 1:
  From Node 0 (Weight: 1)
  From Node 0 (Weight: 0)
Edges leading *to* Node 2:
  From Node 1 (Weight: 2)
  From Node 1 (Weight: 0)
  From Node 0 (Weight: 2)
Edges leading *to* Node 3:
  From Node 2 (Weight: -3)
  From Node 2 (Weight: 0)
  From Node 1 (Weight: -6)
Edges leading *to* Node 4:
  From Node 3 (Weight: 4)
  From Node 3 (Weight: 0)
  From Node 2 (Weight: -12)
Edges leading *to* Node 5:
  From Node 4 (Weight: 5)
  From Node 4 (Weight: 0)
  From Node 3 (Weight: 20)


In [6]:
def find_max_score_in_dag(graph, n):
    """
    Finds the longest path from node 0 to node n in the DAG.
    """
    # max_score[i] stores the longest path value to reach node i
    max_score = [float('-inf')] * (n + 1)
    max_score[0] = 0 # Starting point has a score of 0
    
    # Iterate through nodes in topological order (0 to n)
    for i in range(1, n + 1):
        # Check all incoming edges to node i
        for source_node, weight in graph[i]:
            # Relax the edge: update max_score[i] if a longer path is found
            max_score[i] = max(max_score[i], max_score[source_node] + weight)
            
    return max_score[n]

In [7]:
n = len(pin_values)
max_score_dag = find_max_score_in_dag(dag_graph, n)
print(f"\nThe maximum score found using the DAG algorithm is: {max_score_dag}")


The maximum score found using the DAG algorithm is: 23


### Clearly the DAG is an over engineer version
- But the goal is to show that a problem has different views and could be solve in many ways.
- Showed that dynamic programming shows up in different places